In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import numpy as np
import matplotlib.pyplot as plt

from microsim.population import NHANESDirectSamplePopulation

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

import time

timeI = time.time()

popSize = 600000
nYears = 5

In [2]:
%%time
#35min for 200k
from microsim.treatment_strategy_repository import TreatmentStrategyRepository
from microsim.bp_treatment_strategies import AddASingleBPMedTreatmentStrategy
from microsim.treatment import TreatmentStrategiesType
from microsim.outcome import OutcomeType

controlPop = NHANESDirectSamplePopulation(popSize, 1999)

treatmentStrategies = TreatmentStrategyRepository()
treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()

CPU times: user 3min 44s, sys: 1.27 s, total: 3min 46s
Wall time: 3min 45s


In [3]:
%%time
controlPop.advance_parallel(nYears, None, 5)

CPU times: user 2min, sys: 1min 53s, total: 3min 54s
Wall time: 16min 54s


In [4]:
%%time
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)

CPU times: user 4min 22s, sys: 10.8 s, total: 4min 33s
Wall time: 4min 35s


In [5]:
%%time
treatmentPop.advance_parallel(nYears, treatmentStrategies = treatmentStrategies, nWorkers=5)

CPU times: user 2min 46s, sys: 9min 26s, total: 12min 13s
Wall time: 28min 32s


In [6]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
strokeRR = treatmentRisk/controlRisk

In [7]:
strokeRR

0.8923691402144207

In [8]:
from microsim.cv_model import CVModelBase
import importlib.util
module = importlib.import_module("microsim.cv_model_repository")

In [9]:
interceptStep = 0.5
interceptChange = 0
interceptMale = -11.679980
interceptFemale = -12.823110
strokeRRRef = 0.79

In [10]:
for i in range(20):

    strokeRRDiff = strokeRR-strokeRRRef
    if strokeRRDiff>0:
        interceptChange = -interceptStep
    else:
        interceptChange= interceptStep
    interceptMale += interceptChange
    interceptFemale += interceptChange

    print(interceptMale, interceptFemale, interceptStep, interceptChange, strokeRRDiff, strokeRR)    
    
    class CVModelMale(CVModelBase):
        """CV model details for male gender."""
        def __init__(self):
            maleCVCoefficients = {
            "lagAge": 0.064200,
            "black": 0.482835,
            "lagSbp#lagSbp": -0.000061,
            "lagSbp": 0.038950,
            "current_bp_treatment": 2.055533,
            "current_diabetes": 0.842209,
            "current_smoker": 0.895589,
            "lagAge#black": 0,
            "lagSbp#current_bp_treatment": -0.014207,
            "lagSbp#black": 0.011609,
            "black#current_bp_treatment": -0.119460,
            "lagAge#lagSbp": 0.000025,
            "black#current_diabetes": -0.077214,
            "black#current_smoker": -0.226771,
            "lagSbp#black#current_bp_treatment": 0.004190,
            "lagAge#lagSbp#black": -0.000199,
            "Intercept": interceptMale,
            }
            tot_chol_hdl_ratio=0.193307
            black_race_x_tot_chol_hdl_ratio=-0.117749
            super().__init__(maleCVCoefficients, tot_chol_hdl_ratio, black_race_x_tot_chol_hdl_ratio)

    class CVModelFemale(CVModelBase):
        """CV model details for female gender."""
        def __init__(self):
            femaleCVCoefficients = {
            "lagAge": 0.106501,
            "black": 0.432440,
            "lagSbp#lagSbp": 0.000056,
            "lagSbp": 0.017666,
            "current_bp_treatment": 0.731678,
            "current_diabetes": 0.943970,
            "current_smoker": 1.009790,
            "lagAge#black": -0.008580,
            "lagSbp#current_bp_treatment": -0.003647,
            "lagSbp#black": 0.006208,
            "black#current_bp_treatment": 0.152968,
            "lagAge#lagSbp": -0.000153,
            "black#current_diabetes": 0.115232,
            "black#current_smoker": -0.092231,
            "lagSbp#black#current_bp_treatment": -0.000173,
            "lagAge#lagSbp#black": -0.000094,
            "Intercept": interceptFemale,
            }
            tot_chol_hdl_ratio=0.151318
            black_race_x_tot_chol_hdl_ratio=0.070498
            super().__init__(femaleCVCoefficients, tot_chol_hdl_ratio, black_race_x_tot_chol_hdl_ratio)
        
    setattr(module, "CVModelFemale", CVModelFemale)
    setattr(module, "CVModelMale", CVModelMale)
    
    treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
    treatmentPop.advance(nYears, treatmentStrategies = treatmentStrategies)
    controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
    treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
    strokeRR = treatmentRisk/controlRisk
    if (strokeRR-strokeRRRef)*strokeRRDiff<0:
        interceptStep = interceptStep/2

-12.17998 -13.32311 0.5 -0.5 0.10236914021442067 0.8923691402144207
-11.92998 -13.07311 0.25 0.25 -0.21127391212949354 0.5787260878705065
-11.67998 -12.82311 0.25 0.25 -0.04856632331301247 0.7414336766869876
-11.80498 -12.94811 0.125 -0.125 0.11813537944082386 0.9081353794408239
-11.92998 -13.07311 0.125 -0.125 0.02206642842127382 0.8120664284212739
-11.86748 -13.01061 0.0625 0.0625 -0.022079041412655154 0.7679209585873449
-11.80498 -12.94811 0.0625 0.0625 -0.03364095017868418 0.7563590498213159
-11.83623 -12.97936 0.03125 -0.03125 0.03173638848013438 0.8217363884801344
-11.820605 -12.963735 0.015625 0.015625 -0.004631070002102278 0.7853689299978978
-11.8284175 -12.9715475 0.0078125 -0.0078125 0.039514399831826585 0.8295143998318266
-11.83623 -12.97936 0.0078125 -0.0078125 0.020384696237124134 0.8103846962371242
-11.8440425 -12.9871725 0.0078125 -0.0078125 0.005459323102795843 0.7954593231027959
-11.851855 -12.994985 0.0078125 -0.0078125 0.010294303132226124 0.8002943031322262
-11.8479

In [11]:
timeF = time.time()
(timeF-timeI)/60

1296.5472847660383

In [12]:
strokeRRDiff = strokeRR-strokeRRRef
if strokeRRDiff>0:
    interceptChange = -interceptStep
else:
    interceptChange= interceptStep
interceptMale += interceptChange
interceptFemale += interceptChange

print(interceptMale, interceptFemale, interceptStep, interceptChange, strokeRRDiff, strokeRR)

-11.8469721875 -12.9901021875 0.00048828125 -0.00048828125 0.01134538574731958 0.8013453857473196


In [ ]:
interceptMale = -11.8469721875
interceptFemale = -12.9901021875
interceptChange = -0.00048828125

In [13]:
controlRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                           controlPop._people)))/controlPop._n
treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                             treatmentPop._people)))/treatmentPop._n
miRR = treatmentRiskMI/controlRiskMI
miRR

0.8011792129439189

In [ ]:
%%time
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)

In [ ]:
%%time
treatmentPop.advance(nYears, treatmentStrategies = treatmentStrategies)

In [ ]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
strokeRR = treatmentRisk/controlRisk
strokeRR

In [ ]:
if (strokeRR-strokeRRRef)*strokeRRDiff<0:
    interceptStep = interceptStep/2
strokeRRDiff = strokeRR-strokeRRRef